In [1]:
from meep import mpb
import meep as mp


In [8]:
def compare_lat_bands(**kwargs, debug=False):
    
    
    bands_list=[]
    p=kwargs.get('p',1)
    r = kwargs.get('r',0.3):
    dir=kwargs.get('dir','old')        
    nrows=kwargs.get('nrows',2)
    twod=kwargs.get('twod',False)
    a=422
    
    h_dl = 220/a
        
        # Materials
    Si = mp.Medium(index=3.48) #3.48
    Gamma = mp.Vector3(0, 0)
    M = mp.Vector3(1/2, -1/2)
    
    
        
    # Triangular lattice:
    lattice = mp.Lattice(size=mp.Vector3(1, 1),
                                      basis1=mp.Vector3(math.sqrt(3)/2, 1/2),
                                      basis2=mp.Vector3(math.sqrt(3)/2, -1/2))
        
        # Transform the k-points defined above into the reciprocal space:
    #Gamma = mp.cartesian_to_reciprocal(Gamma, lattice)
    if dir=='old':
        
        K = mp.Vector3(1/2, 0)
        K = mp.cartesian_to_reciprocal(K, lattice)
        Gamma=mp.cartesian_to_reciprocal(K,lattice)
        # Define the path in k-space:
        # k_points
    else:
        K=mp.Vector3(-1.0/3,1.0/3)   
    
    
    if points=='full':
        
        k_points = [Gamma,   K,       # K
                    #mp.Vector3(0, 0.5),  # K
                    #mp.Vector3(-0.5)           # Gamma
                       ]
    else:
        k_points = [K/2,          # Gamma
                        K,       # K
                    #mp.Vector3(0, 0.5),  # K
                    #mp.Vector3(-0.5)           # Gamma
                       ]    
            # Interpolate points in between the high-symmetry points:
    k_points = mp.interpolate(k_resolution, k_points)
            # Simulation cell
         
    if nrows==2 or nrows==1:
        if twod:
            geometry_lattice=mp.Lattice(size=mp.Vector3(2*(np.sqrt(3))*(width-0.5), 1))
        else:
            geometry_lattice=mp.Lattice(size=mp.Vector3(2*(np.sqrt(3))*(width-0.5), 1,2))   
    else:
        if twod:
            geometry_lattice = mp.Lattice(size=mp.Vector3(2*(np.sqrt(3))*(width-0.5)-2*offset1, 1))   
        else:
            
            geometry_lattice = mp.Lattice(size=mp.Vector3(2*(np.sqrt(3))*(width-0.5)-2*offset1, 1,2))
            
            # Geometry
    l1 = mp.Cylinder(center=(-np.sqrt(3)/2, 0), radius=r, height=mp.inf, material=mp.air)
    l2 = mp.Cylinder(center=(-np.sqrt(3), -1/2), radius=r, height=mp.inf, material=mp.air)
    l3 = mp.Cylinder(center=(-np.sqrt(3), 1/2), radius=r, height=mp.inf, material=mp.air)
            
    r1 = mp.Cylinder(center=(np.sqrt(3)/2, 0), radius=r, height=mp.inf, material=mp.air)
    r2 = mp.Cylinder(center=(np.sqrt(3), -1/2), radius=r, height=mp.inf, material=mp.air)
    r3 = mp.Cylinder(center=(np.sqrt(3), 1/2), radius=r, height=mp.inf, material=mp.air)
            
    c1 = mp.Cylinder(center=(0, -1/2), radius=r, height=mp.inf, material=mp.air)
    c2 = mp.Cylinder(center=(0, 1/2), radius=r, height=mp.inf, material=mp.air)
            
    l01=mp.Cylinder(center=(-np.sqrt(3)/2+offset1, 0), radius=r, height=mp.inf, material=mp.air)
    r01=mp.Cylinder(center=(np.sqrt(3)/2-offset1, 0), radius=r, height=mp.inf, material=mp.air)
            
    l02 = mp.Cylinder(center=(-np.sqrt(3)+offset2, -1/2), radius=r, height=mp.inf, material=mp.air)
    l03 = mp.Cylinder(center=(-np.sqrt(3)+offset2, 1/2), radius=r, height=mp.inf, material=mp.air)
            
    r02 = mp.Cylinder(center=(np.sqrt(3)-offset2, -1/2), radius=r, height=mp.inf, material=mp.air)
    r03 = mp.Cylinder(center=(np.sqrt(3)-offset2, 1/2), radius=r, height=mp.inf, material=mp.air)
            
    if twod:        
        dl = mp.Block(size=(mp.inf, mp.inf), center=(0, 0), material=Si, e1=(1, 0), e2=(0, 1))   
    else:
        dl = mp.Block(size=(mp.inf, mp.inf, h_dl), center=(0, 0, 0), material=Si, e1=(1, 0, 0), e2=(0, 1, 0), e3=(0, 0, 1))
            
    geometry = [dl, l01, l02, l03, r01, r02, r03]#, c1, c2]
  
    if nrows==1:
        geometry = [dl, l01, l2, l3, r01, r2, r3]#, c1, c2]    
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(-np.sqrt(3), 0), 1, width, [l1, l2, l3]))
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(np.sqrt(3), 0), 1, width, [r1, r2, r3]))
    if nrows==2:
        geometry = [dl, l01, l02, l03, r01, r02, r03]#, c1, c2] 
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(-np.sqrt(3), 0), 1, width, [l1, l2, l3]))
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(np.sqrt(3), 0), 1, width, [r1, r2, r3]))

    else:
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(-np.sqrt(3), 0), 1, width, [l01, l02, l03]))
        geometry.extend(mp.geometric_objects_duplicates(mp.Vector3(np.sqrt(3), 0), 1, width, [r01, r02, r03]))

            # Mode solver
            
    if debug:
        s = mp.Source(src=mp.GaussianSource(1, fwidth=1), component=mp.Hz, center=mp.Vector3(0.1234, 0.2345))
        if nrows==2 or nrows==1:
            sim = mp.Simulation(cell_size=mp.Vector3(2*(np.sqrt(3))*(width-0.5), 1), geometry=geometry, sources=[], symmetries=[],
                        boundary_layers=[], resolution=resolution, default_material=Si)
            sim.plot2D()
    
            plt.show()       
        else:
            sim = mp.Simulation(cell_size=mp.Vector3(2*(np.sqrt(3))*(width-0.5)-2*offset1, 1), geometry=geometry, sources=[], symmetries=[],
                        boundary_layers=[], resolution=resolution, default_material=Si)
            sim.plot2D()
    
            plt.show()   
    else:        
        ms = mpb.ModeSolver(num_bands=num_bands,
                                k_points=k_points,
                                geometry=geometry,
                                geometry_lattice=geometry_lattice,
                                resolution=resolution,
                                tolerance=1.0e-5,
                                default_material=mp.air)
        ms.run_te()
        print('\n\n')
        print('\n Bands calculated for grid point ', offset1, offset2 )
        
        
        end_time=time.time()

        print('Time taken (min) ', (end_time-start_time)/60)
        
        
        bands=np.transpose(ms.all_freqs)
        
        return bands


1707744031.4693024

'2024-02-12'